In [37]:
import keras
import pandas as pd
import numpy as np
import matchzoo as mz

In [38]:
train_pack = mz.datasets.wiki_qa.load_data('train', task='ranking')
valid_pack = mz.datasets.wiki_qa.load_data('dev', task='ranking', filtered=True)
predict_pack = mz.datasets.wiki_qa.load_data('test', task='ranking', filtered=True)

In [39]:
preprocessor = mz.preprocessors.BasicPreprocessor(fixed_length_left=10, fixed_length_right=100, remove_stop_words=False)
train_pack_processed = preprocessor.fit_transform(train_pack)

Processing text_left with chain_transform of Tokenize => Lowercase => PuncRemoval: 100%|██████████| 2118/2118 [00:00<00:00, 6801.59it/s]
Processing text_right with chain_transform of Tokenize => Lowercase => PuncRemoval: 100%|██████████| 18841/18841 [00:03<00:00, 4714.02it/s]
Building Vocabulary from a datapack.: 100%|██████████| 404415/404415 [00:00<00:00, 2085471.93it/s]
Processing text_left with chain_transform of Tokenize => Lowercase => PuncRemoval: 100%|██████████| 2118/2118 [00:00<00:00, 8451.54it/s]
Processing text_right with chain_transform of Tokenize => Lowercase => PuncRemoval: 100%|██████████| 18841/18841 [00:04<00:00, 4583.85it/s]
Processing text_right with transform: 100%|██████████| 18841/18841 [00:00<00:00, 81816.97it/s]


In [40]:
valid_pack_processed = preprocessor.transform(valid_pack)
predict_pack_processed = preprocessor.transform(predict_pack)

Processing text_left with chain_transform of Tokenize => Lowercase => PuncRemoval: 100%|██████████| 122/122 [00:00<00:00, 3727.86it/s]
Processing text_right with chain_transform of Tokenize => Lowercase => PuncRemoval: 100%|██████████| 1115/1115 [00:00<00:00, 3500.00it/s]
Processing text_right with transform: 100%|██████████| 1115/1115 [00:00<00:00, 65730.35it/s]
Processing text_left with chain_transform of Tokenize => Lowercase => PuncRemoval: 100%|██████████| 237/237 [00:00<00:00, 6728.60it/s]
Processing text_right with chain_transform of Tokenize => Lowercase => PuncRemoval: 100%|██████████| 2300/2300 [00:00<00:00, 4194.91it/s]
Processing text_right with transform: 100%|██████████| 2300/2300 [00:00<00:00, 87617.84it/s]


In [42]:
ranking_task = mz.tasks.Ranking(loss=mz.losses.RankCrossEntropyLoss(num_neg=4))
ranking_task.metrics = [
    mz.metrics.NormalizedDiscountedCumulativeGain(k=3),
    mz.metrics.NormalizedDiscountedCumulativeGain(k=5),
    mz.metrics.MeanAveragePrecision()
]

In [43]:
bin_size = 30
model = mz.models.ANMM()
# model.params['input_shapes'] = [(5,), (5, 30)]
model.params['task'] = ranking_task
model.params['embedding_input_dim'] = preprocessor.context['vocab_size']
model.params['embedding_output_dim'] = 300
model.params['optimizer'] = 'adam'
model.guess_and_fill_missing_params()
model.build()
model.compile()
model.backend.summary()

Parameter "input_shapes" set to [(30,), (30,)].
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
text_right (InputLayer)         (None, 30)           0                                            
__________________________________________________________________________________________________
dropout_9 (Dropout)             (None, 30)           0           text_right[0][0]                 
__________________________________________________________________________________________________
text_left (InputLayer)          (None, 30)           0                                            
__________________________________________________________________________________________________
dense_38 (Dense)                (None, 30)           930         dropout_9[0][0]                  
_____________________________________________________________

In [44]:
glove_embedding = mz.datasets.embeddings.load_glove_embedding(dimension=300)
embedding_matrix = glove_embedding.build_matrix(preprocessor.context['vocab_unit'].state['term_index'])